In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [2]:
with open('dict_tables.pkl', 'rb') as f:
    dict_table = pickle.load(f)
data = pd.read_csv('clean_worldcup_data.csv')
df_fixture = pd.read_csv('clean_worldcup_fixture.csv')

CALCULAR TEAM STRENGHT

In [3]:
#dividir data en ambos quipos
df_home = data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [4]:
df_home= df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'ScoredGoals', 'AwayGoals': 'ConcededGoals'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'ConcededGoals', 'AwayGoals': 'ScoredGoals'})
#renombrando columnas

In [5]:
#concatenar dataframes
team_strength_df = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

agrupando los goles anotados y recibidos por cada seleccion y generando un promedio

## Creando funcion de prediccion

In [6]:
def predict_points(home, away):
    if home in team_strength_df.index and away in team_strength_df.index:
        lambda_home = team_strength_df.at[home, 'ScoredGoals'] * team_strength_df.at[away, 'ConcededGoals']
        lambda_away = team_strength_df.at[away, 'ScoredGoals'] * team_strength_df.at[home, 'ConcededGoals']
        
        prob_home, prob_away, prob_draw = 0, 0, 0
        #simulando posibles resultados y calculando probabilidad
        #calculando (x, y) siendo en la funcion matematica --> (x, lambda)
        for x in range(0, 11): #x --> nro de goles home
            for y in range(0, 11): #y --> nro goles away
                p = poisson.pmf(x, lambda_home) * poisson.pmf(y, lambda_away)
                if x == y:
                    prob_draw += p #probablidad empate
                elif x > y:
                    prob_home += p #probabilidad home gane
                else: #(y>x)
                    prob_away += p #probabilidad away gane
        
        points_home = 3 * prob_home + prob_draw 
        points_away = 3 * prob_away + prob_draw 
        return (points_home, points_away)
    else:
        return (0, 0) #en caso de que se introduzca un equipo q no tengamos data
                
            
        

In [7]:
#testear funcion 
predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar', 'Ecuador')#el primer mundial de Qatar es el de 2002 ( no esta en la data historica)


(0, 0)

#Preparando predicciones 

1. Fase de Grupos

In [8]:
#dividir el fixture en fases, grupos, octavos, cuartos...

df_fixture_groups = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()



In [9]:
dict_table
#df_fixture_groups

{'Group A':    Pos         Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1    Qatar (H)    0  0  0  0   0   0   0    0
 1    2      Ecuador    0  0  0  0   0   0   0    0
 2    3      Senegal    0  0  0  0   0   0   0    0
 3    4  Netherlands    0  0  0  0   0   0   0    0,
 'Group B':    Pos           Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1        England    0  0  0  0   0   0   0    0
 1    2           Iran    0  0  0  0   0   0   0    0
 2    3  United States    0  0  0  0   0   0   0    0
 3    4          Wales    0  0  0  0   0   0   0    0,
 'Group C':    Pos          Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     Argentina    0  0  0  0   0   0   0    0
 1    2  Saudi Arabia    0  0  0  0   0   0   0    0
 2    3        Mexico    0  0  0  0   0   0   0    0
 3    4        Poland    0  0  0  0   0   0   0    0,
 'Group D':    Pos       Team  Pld  W  D  L  GF  GA  GD  Pts
 0    1     France    0  0  0  0   0   0   0    0
 1    2  Australia    0  0  0  0   0   0   0    0
 2    3 

In [10]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group6 = df_fixture_groups[df_fixture_groups['home'].isin(teams_in_group)]
    for index, row in df_fixture_group6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        
        #adicionando los puntos q retorna la funcion a dict_table
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += int(points_home)
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += int(points_away)
        
        #realizando arreglos y ordenando x puntos
        dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
        dict_table[group] = dict_table[group][['Team', 'Pts']]
        dict_table[group] = dict_table[group].round(0) #redondeando puntos x equipo actual

2. Octavos de finales

In [11]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [12]:
# octavos: df_fixture_knockout
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']#ganador del grupo
    runners_up = dict_table[group].loc[1, 'Team']#segundo puesto
    
    #actualizar valores a dict_table
    df_fixture_knockout.replace({f'Winners {group}': group_winner,
                                 f'Runners-up {group}': runners_up}, inplace=True)
    
    df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,France,2022,?
50,Denmark,Match 52,Poland,2022,?
51,England,Match 51,Ecuador,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Portugal,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Uruguay,Match 56,Switzerland,2022,?


funcion get_winner()

In [13]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, home_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [14]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Wales
49,Argentina,Match 50,France,2022,France
50,Denmark,Match 52,Poland,2022,Poland
51,England,Match 51,Ecuador,2022,Ecuador
52,Germany,Match 53,Belgium,2022,Belgium
53,Brazil,Match 54,Portugal,2022,Portugal
54,Croatia,Match 55,Spain,2022,Spain
55,Uruguay,Match 56,Switzerland,2022,Switzerland


3. Cuartos de Finales

In [15]:
df_fixture_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [20]:
#update tables func
def update_table(df_fixture_round1, df_fixture_round2):
    for index, row in df_fixture_round1.iterrows():
        winner = df_fixture_round1.loc[index, 'winner']
        match = df_fixture_round1.loc[index, 'score']
        df_fixture_round2.replace({f'Winners {match}': winner}, inplace=True)
    df_fixture_round2['winner'] = '?'
    return df_fixture_round2
#la funcion toma dos fases ( o rondas) del mundial como parametro
#df_fixture_round1 va a ser la ronda que se quiere tomar de base para actualizar los resultados de df_fixture_round2

In [21]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Belgium,Match 58,Portugal,2022,?
57,Wales,Match 57,France,2022,?
58,Spain,Match 60,Switzerland,2022,?
59,Ecuador,Match 59,Poland,2022,?


In [23]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Belgium,Match 58,Portugal,2022,Portugal
57,Wales,Match 57,France,2022,France
58,Spain,Match 60,Switzerland,2022,Switzerland
59,Ecuador,Match 59,Poland,2022,Poland


4. Semifinales

In [24]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,France,Match 61,Portugal,2022,?
61,Poland,Match 62,Switzerland,2022,?


In [25]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,France,Match 61,Portugal,2022,Portugal
61,Poland,Match 62,Switzerland,2022,Switzerland


5.Final

In [26]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Portugal,Match 64,Switzerland,2022,?


In [27]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Portugal,Match 64,Switzerland,2022,Switzerland
